# Sample from a specific run

Set `RUN_DIR` to the run folder you want (the one containing `.hydra/`).

In [16]:
from pathlib import Path
import sys
from typing import Literal
import torch
from omegaconf import OmegaConf
import hydra

ROOT = Path(".").resolve()
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from discrete_diffusion.data import get_tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_float32_matmul_precision("high")
torch.set_grad_enabled(False)

In [22]:
def load_model_from_run(run_dir: Path, ckpt_type: Literal["best", "last"] = "best"):
    ckpt_path = (
        Path(run_dir) / "dummy_checkpoints" / "checkpoints" / f"{ckpt_type}.ckpt"
    )
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    ckpt_cfg = ckpt["hyper_parameters"]["config"]
    hydra_cfg = OmegaConf.load(Path(run_dir) / ".hydra" / "config.yaml")

    tokenizer = get_tokenizer(ckpt_cfg)
    algo_target = ckpt_cfg.algo._target_
    algo_cls = hydra.utils.get_class(algo_target)
    model = algo_cls.load_from_checkpoint(
        ckpt_path,
        config=ckpt_cfg,
        tokenizer=tokenizer,
        map_location=device,
    )
    model.to(device)
    model.eval()
    return model, tokenizer, ckpt_path, ckpt_cfg, hydra_cfg

In [26]:
# Sample
NUM_SAMPLES = 8
NUM_STEPS = None  # use model default

RUN_DIR = Path("../outputs/roneneldan/TinyStories/2026.02.05/130646")

model, tokenizer, ckpt_path, ckpt_cfg, hydra_cfg = load_model_from_run(RUN_DIR)
print("Checkpoint:", ckpt_path)

samples = model.generate_samples(num_samples=NUM_SAMPLES, num_steps=NUM_STEPS)
texts = tokenizer.batch_decode(samples.detach().cpu(), skip_special_tokens=True)
for i, text in enumerate(texts):
    print(f"--- sample {i} ---")
    print(text)
    print()

/local/home/mboronski/PDiff/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Checkpoint: ../outputs/roneneldan/TinyStories/2026.02.05/130646/dummy_checkpoints/checkpoints/best.ckpt


AssertionError: 